In [1]:
# Import basic libraries
import pandas as pd
import numpy as np

In [14]:
#!pip install matplotlib-venn (uncomment if the module isn't imported)

# Import visualisation libraries
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns
import pandas as pd
import numpy as np
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import ast
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from collections import Counter
import os
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [8]:
ten_tropes_df = pd.read_csv('simpsons_10_tropes.csv')
ten_tropes_binarised_df = pd.read_csv('simpsons_10_tropes_binarised.csv')
tropes_binarised_df = pd.read_csv('10_tropes_binarised.csv')

In [4]:
ten_tropes_df

,Episode Title,Full Story,text_lemma,10 Tropes
0,Simpsons Roasting on an Open Fire,Homer hastily drives the Family Sedan with Mar...,homer hastily drive the family sedan with marg...,"['Running Gag', 'Establishing Character Moment']"
1,Bart the Genius,The Simpson family is playing Scrabble in the ...,the simpson family be play scrabble in the liv...,"['Imagine Spot', 'Comically Missing the Point'..."
2,Homers Odyssey,The episode begins in front of Springfield Ele...,the episode begin in front of springfield elem...,"['Karma Houdini', 'Characterization Marches On..."
3,Theres No Disgrace Like Home,Bart and Lisa are fighting but it is not long ...,bart and lisa be fight but it be not long unti...,"['Imagine Spot', 'Characterization Marches On'..."
4,Bart the General,Bart and Lisa fight over Lisas cupcakesThe epi...,bart and lisa fight over lisas cupcakesthe epi...,"['Imagine Spot', 'Characterization Marches On'..."
5,Moaning Lisa,Lisa feels depressedThe episode starts with a ...,lisa feel depressedthe episode start with a me...,"['Characterization Marches On', 'Comically Mis..."
6,The Call of the Simpsons,The episode begins with Homer and Bart outside...,the episode begin with homer and bart outside ...,"['Comically Missing the Point', 'Couch Gag', '..."
7,The Telltale Head,Homer and Bart being chased by the mobThe epis...,homer and bart be chase by the mobthe episode ...,['Couch Gag']
8,Life on the Fast Lane,It is Marges birthday so Bart and Lisa prepare...,it be marge birthday so bart and lisa prepare ...,"['Couch Gag', 'Special Guest', 'Establishing C..."
9,Homers Night Out,Bart purchases a miniature spy camera from a m...,bart purchase a miniature spy camera from a ma...,"['Karma Houdini', 'Comically Missing the Point..."


In [5]:
ten_tropes_binarised_df

,Episode Title,Full Story,text_lemma,Catchphrase,Characterization Marches On,Comically Missing the Point,Couch Gag,Disproportionate Retribution,Establishing Character Moment,Imagine Spot,Karma Houdini,Running Gag,Special Guest
0,Simpsons Roasting on an Open Fire,Homer hastily drives the Family Sedan with Mar...,homer hastily drive the family sedan with marg...,0,0,0,0,0,1,0,0,1,0
1,Bart the Genius,The Simpson family is playing Scrabble in the ...,the simpson family be play scrabble in the liv...,1,0,1,1,0,1,1,0,0,0
2,Homers Odyssey,The episode begins in front of Springfield Ele...,the episode begin in front of springfield elem...,0,1,1,1,0,0,0,1,0,1
3,Theres No Disgrace Like Home,Bart and Lisa are fighting but it is not long ...,bart and lisa be fight but it be not long unti...,1,1,1,1,1,0,1,0,1,0
4,Bart the General,Bart and Lisa fight over Lisas cupcakesThe epi...,bart and lisa fight over lisas cupcakesthe epi...,0,1,0,1,1,0,1,0,1,0
5,Moaning Lisa,Lisa feels depressedThe episode starts with a ...,lisa feel depressedthe episode start with a me...,0,1,1,1,0,0,0,0,0,1
6,The Call of the Simpsons,The episode begins with Homer and Bart outside...,the episode begin with homer and bart outside ...,0,0,1,1,0,0,0,0,0,1
7,The Telltale Head,Homer and Bart being chased by the mobThe epis...,homer and bart be chase by the mobthe episode ...,0,0,0,1,0,0,0,0,0,0
8,Life on the Fast Lane,It is Marges birthday so Bart and Lisa prepare...,it be marge birthday so bart and lisa prepare ...,0,0,0,1,0,1,0,0,0,1
9,Homers Night Out,Bart purchases a miniature spy camera from a m...,bart purchase a miniature spy camera from a ma...,0,0,1,1,0,0,0,1,0,1


In [9]:
tropes_binarised_df

,Catchphrase,Characterization Marches On,Comically Missing the Point,Couch Gag,Disproportionate Retribution,Establishing Character Moment,Imagine Spot,Karma Houdini,Running Gag,Special Guest,Trope Description,Text Length,lemma
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Okay Ralphie You win this time But well be ba...,917,okay ralphie you win this time but well be bac...
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,After seven seasons weve pretty much said ever...,864,after seven season weve pretty much say everyt...
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Catchphrase may refer to one of the following ...,642,catchphrase may refer to one of the follow cha...
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I started to walk down the street when I heard...,1507,i start to walk down the street when i heard a...
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Thats not a running gag Thats a pun Its a runn...,1633,thats not a run gag thats a pun it a run gag n...
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,But that was two years agoRevenge is a dish be...,2243,but that be two year agorevenge be a dish best...
6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,In just a handful of scenes weve established t...,2597,in just a handful of scene weve establish the ...
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,So is Satan just generous when people dont hav...,5204,so be satan just generous when people dont hav...
8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Nothing reconciles a past of animal abuse bett...,1872,nothing reconciles a past of animal abuse well...
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Special but unknown guest starnote How can you...,728,special but unknown guest starnote how can you...


In [10]:
ten_tropes_df.describe

<bound method NDFrame.describe of                         Episode Title  \
0   Simpsons Roasting on an Open Fire   
1                     Bart the Genius   
2                      Homers Odyssey   
3        Theres No Disgrace Like Home   
4                    Bart the General   
5                        Moaning Lisa   
6            The Call of the Simpsons   
7                   The Telltale Head   
8               Life on the Fast Lane   
9                    Homers Night Out   
10                The Crepes of Wrath   
11                 Krusty Gets Busted   
12             Some Enchanted Evening   

                                           Full Story  \
0   Homer hastily drives the Family Sedan with Mar...   
1   The Simpson family is playing Scrabble in the ...   
2   The episode begins in front of Springfield Ele...   
3   Bart and Lisa are fighting but it is not long ...   
4   Bart and Lisa fight over Lisas cupcakesThe epi...   
5   Lisa feels depressedThe episode starts with a

In [11]:
ten_tropes_df.columns

Index(['Episode Title', 'Full Story', 'text_lemma', '10 Tropes'], dtype='object')

In [6]:
ten_tropes_binarised_df.describe

<bound method NDFrame.describe of                         Episode Title  \
0   Simpsons Roasting on an Open Fire   
1                     Bart the Genius   
2                      Homers Odyssey   
3        Theres No Disgrace Like Home   
4                    Bart the General   
5                        Moaning Lisa   
6            The Call of the Simpsons   
7                   The Telltale Head   
8               Life on the Fast Lane   
9                    Homers Night Out   
10                The Crepes of Wrath   
11                 Krusty Gets Busted   
12             Some Enchanted Evening   

                                           Full Story  \
0   Homer hastily drives the Family Sedan with Mar...   
1   The Simpson family is playing Scrabble in the ...   
2   The episode begins in front of Springfield Ele...   
3   Bart and Lisa are fighting but it is not long ...   
4   Bart and Lisa fight over Lisas cupcakesThe epi...   
5   Lisa feels depressedThe episode starts with a

In [7]:
ten_tropes_binarised_df.columns

Index(['Episode Title', 'Full Story', 'text_lemma', 'Catchphrase',
       'Characterization Marches On', 'Comically Missing the Point',
       'Couch Gag', 'Disproportionate Retribution',
       'Establishing Character Moment', 'Imagine Spot', 'Karma Houdini',
       'Running Gag', 'Special Guest'],
      dtype='object')

In [12]:
tropes_binarised_df.describe

<bound method NDFrame.describe of    Catchphrase  Characterization Marches On  Comically Missing the Point  \
0          0.0                          0.0                          0.0   
1          0.0                          0.0                          0.0   
2          1.0                          0.0                          0.0   
3          0.0                          0.0                          1.0   
4          0.0                          0.0                          0.0   
5          0.0                          0.0                          0.0   
6          0.0                          0.0                          0.0   
7          0.0                          0.0                          0.0   
8          0.0                          1.0                          0.0   
9          0.0                          0.0                          0.0   

   Couch Gag  Disproportionate Retribution  Establishing Character Moment  \
0        0.0                           0.0          

In [13]:
tropes_binarised_df.columns

Index(['Catchphrase', 'Characterization Marches On',
       'Comically Missing the Point', 'Couch Gag',
       'Disproportionate Retribution', 'Establishing Character Moment',
       'Imagine Spot', 'Karma Houdini', 'Running Gag', 'Special Guest',
       'Trope Description', 'Text Length', 'lemma'],
      dtype='object')

In [ ]:
# Define the model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(100,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
model.evaluate(x_test, y_test)